In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import re
import csv

# 웹 드라이버 설정 및 옵션
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# 드라이버 객체 생성
driver = webdriver.Chrome(options=options)
driver.get("https://www.sellcarauction.co.kr/newfront/successfulbid/sb/front_successfulbid_sb_list.do")

# 모든 데이터를 저장할 리스트
all_car_data = []

def extract_data_from_page():
    """
    현재 페이지에서 데이터를 추출하고 all_car_data 리스트에 추가합니다.
    """
    try:
        # 페이지 로딩 후 HTML 소스 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        table_body = soup.find('tbody', class_='text-center text_vert_midd')
        if not table_body:
            return 0
        
        rows = table_body.find_all('tr', role='row')
        current_page_data = []
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 7:
                car_details_html = cols[2]
                model_name = car_details_html.find('strong').get_text(strip=True)
                
                next_sibling = car_details_html.find('br').next_sibling
                other_details_text = next_sibling.get_text(strip=True) if next_sibling else ""
                other_details_text = re.sub(r'\s{2,}', ' ', other_details_text)
                other_details_parts = [part.strip() for part in other_details_text.split('|')]
                
                while len(other_details_parts) < 5:
                    other_details_parts.append('')
                
                car_info = {
                    '경매일': cols[0].get_text(strip=True),
                    '경매장': cols[1].get_text(strip=True),
                    '차량정보': model_name,
                    '연식': other_details_parts[0],
                    '변속기': other_details_parts[1],
                    '연료': other_details_parts[2],
                    '배기량': other_details_parts[3],
                    '색상': other_details_parts[4],
                    '용도': cols[3].get_text(strip=True),
                    '주행거리': cols[4].get_text(strip=True),
                    '평가': cols[5].get_text(strip=True),
                    '낙찰가(만원)': cols[6].find('strong').get_text(strip=True).replace(',', '')
                }
                current_page_data.append(car_info)
        
        all_car_data.extend(current_page_data)
        return len(current_page_data)
    
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생: {e}")
        return 0

try:
    # "검색" 버튼이 페이지에 나타날 때까지 명시적으로 대기
    search_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "a.button.btn_small.btn_search"))
    )
    print("✔️ '검색' 버튼을 찾았습니다. 클릭을 시도합니다.")
    search_button.click()
    
    # 첫 페이지가 로드될 때까지 기다림
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "tbody.text-center.text_vert_midd"))
    )

    page_block_num = 1
    while True:
        print(f"\n--- 페이지 블록 {page_block_num} 시작 ---")

        # 현재 페이지의 데이터를 추출
        num_items = extract_data_from_page()
        current_page_number = int(driver.find_element(By.CSS_SELECTOR, "ul.pagination li.active a").text)
        print(f"✔️ {current_page_number} 페이지의 {num_items}개 차량 정보를 수집했습니다. 총 {len(all_car_data)}개.")

        # 다음 페이지 버튼을 찾아서 클릭
        try:
            next_page_number = current_page_number + 1
            next_page_locator = By.XPATH, f"//ul[contains(@class, 'pagination')]//a[text()='{next_page_number}']"
            
            # 다음 페이지 버튼이 클릭 가능할 때까지 기다림
            next_page_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable(next_page_locator)
            )
            
            next_page_button.click()
            print(f"✔️ {next_page_number} 페이지로 이동합니다.")
            
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "tbody.text-center.text_vert_midd"))
            )
            
        except (NoSuchElementException, TimeoutException):
            # 다음 페이지 번호 버튼이 없으면, '>' 버튼을 찾아 다음 페이지 블록으로 이동
            try:
                next_block_button_locator = By.XPATH, "//ul[contains(@class, 'pagination')]//a[text()='>']"
                next_block_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable(next_block_button_locator)
                )
                
                next_block_button.click()
                print("\n✔️ 다음 페이지 블록으로 이동합니다.")
                page_block_num += 1
                
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "tbody.text-center.text_vert_midd"))
                )

            except (NoSuchElementException, TimeoutException):
                print("\n⭐ 마지막 페이지에 도달했거나 다음 페이지 버튼을 찾을 수 없습니다. 크롤링을 종료합니다.")
                break

except (NoSuchElementException, TimeoutException) as e:
    print(f"❌ 크롤링 중 오류 발생: {e}")
    
finally:
    fieldnames = ['경매일', '경매장', '차량정보', '연식', '변속기', '연료', '배기량', '색상', '용도', '주행거리', '평가', '낙찰가(만원)']
    with open('car_data.csv', 'w', encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(all_car_data)
        print(f"✔️ 총 {len(all_car_data)}개의 수집된 데이터를 'car_data.csv' 파일에 성공적으로 저장했습니다.")
    
    driver.quit()
    print("✔️ 웹 드라이버가 종료되었습니다.")


In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import re
import csv
import sys

# --- 블로그 정보 설정 ---
blog_id = 'hdshin7'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'

# --- 웹 드라이버 설정 ---
def get_driver():
    """
    주어진 옵션으로 Selenium 드라이버를 초기화하는 함수
    """
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 브라우저 창을 띄우지 않는 모드
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f'user-agent={user_agent}')
    
    try:
        service = Service()
        return webdriver.Chrome(service=service, options=options)
    except Exception as e:
        print(f"드라이버 초기화 실패: {e}")
        print("크롬 드라이버가 올바르게 설치되었는지 확인하거나, 드라이버 경로를 Service()에 직접 지정해주세요.")
        return None

def get_total_posts(driver, blog_id):
    """
    Selenium을 사용하여 블로그의 전체 게시글 수를 파악합니다.
    """
    if not driver:
        return 0
        
    driver.get(f'https://blog.naver.com/{blog_id}')
    total_posts = 0
    try:
        # iframe으로 전환
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "mainFrame")))
        
        # iframe 내부 HTML 소스를 가져옴
        iframe_html = driver.page_source
        soup = BeautifulSoup(iframe_html, 'html.parser')
        
        # '총 게시글' 텍스트를 포함하는 strong 태그를 찾음
        total_post_tag = soup.find('strong', class_='total_post_count') or soup.find('strong', class_='total_post')

        if total_post_tag:
            total_posts_text = total_post_tag.text.strip().replace(',', '')
            match = re.search(r'\d+', total_posts_text)
            if match:
                total_posts = int(match.group())
        else:
            print("게시글 수를 포함하는 요소를 찾을 수 없습니다.")

    except Exception as e:
        print(f"총 게시글 수 파악 실패: {e}")
    finally:
        driver.switch_to.default_content() # 원래 프레임으로 전환
    
    return total_posts

def get_all_post_urls(driver, blog_id, total_posts):
    """
    총 게시글 수를 기반으로 모든 페이지를 순회하며 게시글 URL을 추출합니다.
    """
    if total_posts == 0:
        return []
        
    print("페이지별 게시글 URL 목록을 수집 중...")
    
    post_urls = set()
    num_pages = (total_posts // 10) + 1
    base_url = f"https://blog.naver.com/PostList.naver?blogId={blog_id}"

    for page_num in range(1, num_pages + 1):
        prev_page_num = ((page_num - 1) // 10) * 10 + 1
        page_url = f"{base_url}&currentPage={page_num}&prevPage={prev_page_num}"
        
        try:
            print(f"페이지 {page_num}/{num_pages}에서 URL 수집 중...")
            driver.get(page_url)
            
            # 게시글 링크가 로드될 때까지 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="logNo"]'))
            )
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            links = soup.find_all('a', href=re.compile(r'logNo=\d+'))
            for link in links:
                href = link.get('href')
                if href and 'logNo=' in href:
                    full_url = f"https://blog.naver.com{href}"
                    post_urls.add(full_url)
        except Exception as e:
            print(f"페이지 {page_num} 로드 실패: {e}")
            continue

    return list(post_urls)

def save_urls_to_csv(urls, filename='blog_urls.csv'):
    """
    추출된 URL 목록을 CSV 파일로 저장합니다.
    """
    if not urls:
        print("저장할 URL이 없습니다.")
        return

    with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['url'])  # 헤더 작성
        for url in urls:
            writer.writerow([url])
    print(f"총 {len(urls)}개의 URL이 '{filename}' 파일에 성공적으로 저장되었습니다.")

if __name__ == '__main__':
    start_time = time.time()

    driver = get_driver()
    if not driver:
        sys.exit(1)

    total_posts = get_total_posts(driver, blog_id)
    
    if total_posts == 0:
        print("총 게시글 수를 찾을 수 없어 스크래핑을 중단합니다.")
        driver.quit()
        sys.exit(1)
    
    print(f"총 게시글 수: {total_posts}개")
    
    post_urls = get_all_post_urls(driver, blog_id, total_posts)
    
    driver.quit()

    if post_urls:
        save_urls_to_csv(post_urls)
    else:
        print("게시글 URL을 찾을 수 없습니다.")
    
    end_time = time.time()
    print(f"총 소요 시간: {end_time - start_time:.2f}초")


게시글 수를 포함하는 요소를 찾을 수 없습니다.
총 게시글 수를 찾을 수 없어 스크래핑을 중단합니다.


SystemExit: 1

/Users/admin/softeer_de_wiki/mission/W2/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
